In [ ]:
### 
#   Regenerate pure day price file
### 


# 讀取 CSV File

def PureController(targets):
    
    import pandas as pd # 引用套件並縮寫為 pd  

    import csv
    import os
    from os import listdir
    from os.path import isfile, join
    from datetime import date

    print ('Start Targets: ',targets)
    today = str(date.today().year).zfill(4)+str(date.today().month).zfill(2)+str(date.today().day).zfill(2)
    ## Template today string for test
    # today = '20200903'

    # 今天抓到的清單
    index_list = [ f[:-4] for f in listdir(join('data', today)) if f[-4:] == '.csv' ]

    # 刪除重複的資料並重新排序
    for stock_id in index_list:
        # stock_id = '2330'
        print ('Stock ID: ',stock_id)

        f = open(join('data', today, stock_id+'.csv'), 'rt')

        colnames=['時間', '即時價位', '即時量', '總量', '5檔賣價', '5檔賣量', '5檔買價', '5檔買量'] 
        df = pd.read_csv(f,names=colnames, header=None)  # header=None, usecols=[0,1,2,3,4,5,6,7])

        result_df = df.drop_duplicates(subset=['時間', '總量'], keep='first')
        result_df.reset_index(inplace=False)


        result_df['Sale5']  = df['5檔賣價'].str.split('_')  
        result_df['Sale5V'] = df['5檔賣量'].str.split('_')
        result_df['Buy5']   = df['5檔買價'].str.split('_')
        result_df['Buy5V']  = df['5檔買量'].str.split('_')

        result_df = result_df.drop(columns=['5檔賣價', '5檔賣量', '5檔買價', '5檔買量'])
        result_df.reset_index(inplace=True)

        last_v = 0 
        last_p = result_df.loc[0,'Sale5'][0]

        for i in range(0, len(result_df)): 
            if int(result_df.loc[i,'總量']) <= last_v  :
                result_df = result_df.drop([i])

                continue 
            else: 
                if (str(result_df.loc[i,'即時價位']) == '-'):
                    result_df.loc[i, '即時價位'] = last_p
                else:
                    last_p = float(result_df.loc[i,'即時價位'])
                if (str(result_df.loc[i,'即時量']) == '-'):
                    result_df.loc[i, '即時量'  ] = int(result_df.loc[i, '總量']) -  last_v
                last_v = result_df.loc[i,'總量']  

        result_df.reset_index(inplace=True)


        # print ('Final ',result_df.head(10))
        f = join('data', today, stock_id+'._csv')  # template CSV File 
        result_df.to_csv(f, header=0,index=False) 

    print ('All Done! ') 
# 


#
#  Main loop 
#
import datetime
import requests
import sched
import time as tm
import json
from time import gmtime, strftime                
                
s = sched.scheduler(tm.time, tm.sleep)


def main_loop ():        
    targets = [_.strip() for _ in open('stocknumber.csv', 'r')]
    
    time = datetime.datetime.now()  
    print("開始更新時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second))

    start_time = datetime.datetime.strptime(str(time.date())+'9:00', '%Y-%m-%d%H:%M')
    end_time =  datetime.datetime.strptime(str(time.date())+'13:35', '%Y-%m-%d%H:%M')
    
    # tm.sleep (3) # 避免證交所伺服器鎖 IP，可能為都是網頁伺服器的rate limiting 在作祟。
    # 判斷爬蟲終止條件
    sleeptimer = 10.5
    if time >= start_time and time <= end_time:
        tm.sleep (sleeptimer)
        try: 
            PureController(targets)
            
        except:
            msg = "證交所網路忙碌！"
        else:
            msg = "更新時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second)
        finally:
            # print("更新時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second))
            print("Done.", msg)
            s.enter(1, 0, main_loop, argument=())
    else:
        print ('非營業時間，不提供連續資料。')
        print ('繼續等待交易時間。。。')
        s.enter(1, 0, main_crawler, argument=())        

'''
def main():
    targets = [_.strip() for _ in open('stocknumber.csv', 'r')]

    controller = CrawlerController(targets)
    data = controller.run()

    recorder = Recorder()
    recorder.record_to_csv(data)
    
'''

if __name__ == '__main__':
    
    s.enter(1, 0, main_loop, argument=())
    s.run()


In [1]:
### 
#   Regenerate pure day price file
### 


# 讀取 CSV File

def PureController(targets):
    
    import pandas as pd # 引用套件並縮寫為 pd  

    import csv
    import os
    from os import listdir
    from os.path import isfile, join
    from datetime import date

    print ('Start Targets: ',targets)
    today = str(date.today().year).zfill(4)+str(date.today().month).zfill(2)+str(date.today().day).zfill(2)
    ## Template today string for test
    # today = '20200903'

    # 今天抓到的清單
    index_list = [ f[:-4] for f in listdir(join('data', today)) if f[-4:] == '.csv' ]

    # 刪除重複的資料並重新排序
    for stock_id in index_list:
        # stock_id = '2330'
        print ('Stock ID: ',stock_id)

        f = open(join('data', today, stock_id+'.csv'), 'rt')

        colnames=['時間', '即時價位', '即時量', '總量', '5檔賣價', '5檔賣量', '5檔買價', '5檔買量'] 
        df = pd.read_csv(f,names=colnames, header=None)  # header=None, usecols=[0,1,2,3,4,5,6,7])

        result_df = df.drop_duplicates(subset=['時間', '總量'], keep='first')
        result_df.reset_index(inplace=False)


        result_df['Sale5']  = df['5檔賣價'].str.split('_')  
        result_df['Sale5V'] = df['5檔賣量'].str.split('_')
        result_df['Buy5']   = df['5檔買價'].str.split('_')
        result_df['Buy5V']  = df['5檔買量'].str.split('_')

        result_df = result_df.drop(columns=['5檔賣價', '5檔賣量', '5檔買價', '5檔買量'])
        result_df.reset_index(inplace=True)

        last_v = 0 
        last_p = result_df.loc[0,'Sale5'][0]

        for i in range(0, len(result_df)): 
            if int(result_df.loc[i,'總量']) <= last_v  :
                result_df = result_df.drop([i])

                continue 
            else: 
                if (str(result_df.loc[i,'即時價位']) == '-'):
                    result_df.loc[i, '即時價位'] = last_p
                else:
                    last_p = float(result_df.loc[i,'即時價位'])
                if (str(result_df.loc[i,'即時量']) == '-'):
                    result_df.loc[i, '即時量'  ] = int(result_df.loc[i, '總量']) -  last_v
                last_v = result_df.loc[i,'總量']  

        result_df.reset_index(inplace=True)


        # print ('Final ',result_df.head(10))
        f = join('data', today, stock_id+'._csv')  # template CSV File 
        result_df.to_csv(f, header=0,index=False) 

    print ('All Done! ') 
# 


#
#  Main loop 
#
import datetime
import requests
import sched
import time as tm
import json
from time import gmtime, strftime                
                
s = sched.scheduler(tm.time, tm.sleep)


def main_loop ():        
    targets = [_.strip() for _ in open('stocknumber.csv', 'r')]
    
    time = datetime.datetime.now()  
    print("統整開始時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second))

    start_time = datetime.datetime.strptime(str(time.date())+'9:00:00', '%Y-%m-%d%H:%M:%S')
    end_time =  datetime.datetime.strptime(str(time.date())+'13:31:00', '%Y-%m-%d%H:%M:%S')
    
    # tm.sleep (3) # 避免證交所伺服器鎖 IP，可能為都是網頁伺服器的rate limiting 在作祟。
    # 判斷爬蟲終止條件
    sleeptimer = 3.5
    if time >= start_time and time <= end_time:
        print ('Starting....')
        tm.sleep (sleeptimer)
        PureController(targets)
        try:
            PureController(targets)
            
        except:
            msg = "無本日資料！"
        else:
            msg = "更新時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second)
        finally:
            # print("更新時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second))
            print("Done.", msg)
            s.enter(1, 0, main_loop, argument=())
    else:
        print ('非營業時間，不提供連續資料。')
        print ('繼續等待交易時間。。。')
        s.enter(1, 0, main_loop, argument=())        

def fivemin_loop(): 
    
    targets = [_.strip() for _ in open('stocknumber.csv', 'r')]  
    time = datetime.datetime.now()  
    
    print("統整五分線開始時間:" + str(time.date())+':'+str(time.hour)+":"+str(time.minute)+":"+str(time.second))

    start_time = datetime.datetime.strptime(str(time.date())+'9:00:00', '%Y-%m-%d%H:%M:%S')
    end_time =  datetime.datetime.strptime(str(time.date())+'13:31:00', '%Y-%m-%d%H:%M:%S')
    ptr_time = start_time  
    while (ptr_time < time):
        print (ptr_time)
        ptr_time  = ptr_time + datetime.timedelta(0,600)
        for i in range (1): 
            print ("Null")
        
def main():
    PureController([])
  

if __name__ == '__main__':
    
    
    fivemin_loop()
    s.enter(1, 0, main_loop, argument=())
    s.enter(120, 0, fivemin_loop, argument=())
    s.run()




統整五分線開始時間:2020-10-30:13:36:33
2020-10-30 09:00:00
Null
2020-10-30 09:10:00
Null
2020-10-30 09:20:00
Null
2020-10-30 09:30:00
Null
2020-10-30 09:40:00
Null
2020-10-30 09:50:00
Null
2020-10-30 10:00:00
Null
2020-10-30 10:10:00
Null
2020-10-30 10:20:00
Null
2020-10-30 10:30:00
Null
2020-10-30 10:40:00
Null
2020-10-30 10:50:00
Null
2020-10-30 11:00:00
Null
2020-10-30 11:10:00
Null
2020-10-30 11:20:00
Null
2020-10-30 11:30:00
Null
2020-10-30 11:40:00
Null
2020-10-30 11:50:00
Null
2020-10-30 12:00:00
Null
2020-10-30 12:10:00
Null
2020-10-30 12:20:00
Null
2020-10-30 12:30:00
Null
2020-10-30 12:40:00
Null
2020-10-30 12:50:00
Null
2020-10-30 13:00:00
Null
2020-10-30 13:10:00
Null
2020-10-30 13:20:00
Null
2020-10-30 13:30:00
Null
統整開始時間:2020-10-30:13:36:34
非營業時間，不提供連續資料。
繼續等待交易時間。。。
統整開始時間:2020-10-30:13:36:35
非營業時間，不提供連續資料。
繼續等待交易時間。。。
統整開始時間:2020-10-30:13:36:36
非營業時間，不提供連續資料。
繼續等待交易時間。。。
統整開始時間:2020-10-30:13:36:37
非營業時間，不提供連續資料。
繼續等待交易時間。。。
統整開始時間:2020-10-30:13:36:38
非營業時間，不提供連續資料。
繼續等待交易時間。。。


KeyboardInterrupt: 